## Fuzzy logic model related with environment and tourism
##### Authors: Alejandra Palacio, José Manuel Ramírez, Juan José Wilches y Carlos Gustavo Vélez.

In [1]:
# To import libraries
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
file = 'Data Fuzzy Models.xlsx'

df = pd.read_excel(file, sheet_name= 'Enviromental')

data_dicts = df.to_dict(orient='records')

commune_data = {row['Código']: {
                            'Nombre': row['Nombre'],
                            'Identificador': row['Identificador'],
                            'Indicador turismo': row['Indicador turismo'],
                            'Corredores verdes': row['Corredores verdes'],
                            'Calidad de Aire': row['Calidad de Aire'],
                            'Producción Agua Potable': row['Producción Agua Potable'],
                            'Turismo sostenible': row.get('Turismo sostenible', None)}  
               for index, row in df.iterrows()}

In [3]:
# Universe of discourse
x_air_quality = np.arange(0, 1.01, 0.01)
x_potable_water = np.arange(0, 1.01, 0.01)
x_green_corridors = np.arange(0, 1.01, 0.01)
x_tourism_indicator = np.arange(0, 1.01, 0.01)
x_sustainable_tourism = np.arange(0, 1.01, 0.01)

# Membership functions for Air Quality

# Low air quality uses a Gaussian function centered at 0, with a small spread
air_quality_low = fuzz.gaussmf(x_air_quality, 0, 0.1)
# Mid air quality uses a trapezoidal function spanning a middle range
air_quality_mid = fuzz.trapmf(x_air_quality, [0.1, 0.15, 0.2, 0.25])
# High air quality uses a Gaussian function centered at 1, with a narrow spread
air_quality_high = fuzz.gaussmf(x_air_quality, 1, 0.25)

# Membership functions for Potable Water Production

# Low potable water production is represented by a triangular function
potable_water_low = fuzz.trimf(x_potable_water, [0, 0, 0.2])
# Mid potable water production is also a triangular function with different bounds
potable_water_mid = fuzz.trimf(x_potable_water, [0, 0.2, 0.4])
# High potable water production is a trapezoidal function covering the upper range
potable_water_high = fuzz.trapmf(x_potable_water, [0.2, 0.4, 1, 1])

# Membership functions for Green Corridors

# Low green corridors uses a Gaussian function centered at 0, representing lower values
green_corridors_low = fuzz.gaussmf(x_green_corridors, 0, 0.3)
# Mid green corridors uses a trapezoidal function for mid-range values
green_corridors_mid = fuzz.trapmf(x_green_corridors, [0.2, 0.4, 0.6, 0.8])
# High green corridors is represented by a triangular function, peaking at 1
green_corridors_high = fuzz.trimf(x_green_corridors, [0.6, 1, 1])

# Membership functions for Tourism Indicator

# Low tourism indicator uses a trapezoidal function for lower values
tourism_indicator_low = fuzz.trapmf(x_tourism_indicator, [0, 0, 0.3, 0.5])
# Mid tourism indicator is represented by a Gaussian function centered at 0.5
tourism_indicator_mid = fuzz.gaussmf(x_tourism_indicator, 0.5, 0.15)
# High tourism indicator is a trapezoidal function for upper values
tourism_indicator_high = fuzz.trapmf(x_tourism_indicator, [0.5, 0.7, 1, 1])

# Membership functions for Sustainable Tourism (output variable)

# Low sustainable tourism uses a Gaussian function centered at 0
sustainable_tourism_low = lambda x: fuzz.gaussmf(x, 0, 0.2)
# Mid sustainable tourism uses a triangular function for the middle range
sustainable_tourism_mid = lambda x: fuzz.gaussmf(x, 0.5, 0.15)
# High sustainable tourism is represented by a Gaussian function centered at 1
sustainable_tourism_high = lambda x: fuzz.gaussmf(x, 1, 0.2)

In [4]:
def rules_base(input_air, input_water, input_corridors, input_tourism):

     # Membership functions with scikit-fuzzy interpolation to create rules

    # Air Quality Membership
    air_fit_low = fuzz.interp_membership(x_air_quality, air_quality_low, input_air)
    air_fit_mid = fuzz.interp_membership(x_air_quality, air_quality_mid, input_air)
    air_fit_high = fuzz.interp_membership(x_air_quality, air_quality_high, input_air)

    # Potable Water Membership
    water_fit_low = fuzz.interp_membership(x_potable_water, potable_water_low, input_water)
    water_fit_mid = fuzz.interp_membership(x_potable_water, potable_water_mid, input_water)
    water_fit_high = fuzz.interp_membership(x_potable_water, potable_water_high, input_water)

    # Green Corridors Membership
    corridors_fit_low = fuzz.interp_membership(x_green_corridors, green_corridors_low, input_corridors)
    corridors_fit_mid = fuzz.interp_membership(x_green_corridors, green_corridors_mid, input_corridors)
    corridors_fit_high = fuzz.interp_membership(x_green_corridors, green_corridors_high, input_corridors)

    # Tourism Indicator Membership
    tourism_fit_low = fuzz.interp_membership(x_tourism_indicator, tourism_indicator_low, input_tourism)
    tourism_fit_mid = fuzz.interp_membership(x_tourism_indicator, tourism_indicator_mid, input_tourism)
    tourism_fit_high = fuzz.interp_membership(x_tourism_indicator, tourism_indicator_high, input_tourism)

    rule1 = air_fit_low * water_fit_low * corridors_fit_low * tourism_fit_low
    rule2 = air_fit_low * water_fit_low * corridors_fit_low * tourism_fit_mid
    rule3 = air_fit_low * water_fit_low * corridors_fit_low * tourism_fit_high
    rule4 = air_fit_low * water_fit_low * corridors_fit_mid * tourism_fit_low
    rule5 = air_fit_low * water_fit_low * corridors_fit_mid * tourism_fit_mid
    rule6 = air_fit_low * water_fit_low * corridors_fit_mid * tourism_fit_high
    rule7 = air_fit_low * water_fit_low * corridors_fit_high * tourism_fit_low
    rule8 = air_fit_low * water_fit_low * corridors_fit_high * tourism_fit_mid
    rule9 = air_fit_low * water_fit_low * corridors_fit_high * tourism_fit_high
    rule10 = air_fit_low * water_fit_mid * corridors_fit_low * tourism_fit_low
    rule11 = air_fit_low * water_fit_mid * corridors_fit_low * tourism_fit_mid
    rule12 = air_fit_low * water_fit_mid * corridors_fit_low * tourism_fit_high
    rule13 = air_fit_low * water_fit_mid * corridors_fit_mid * tourism_fit_low
    rule14 = air_fit_low * water_fit_mid * corridors_fit_mid * tourism_fit_mid
    rule15 = air_fit_low * water_fit_mid * corridors_fit_mid * tourism_fit_high
    rule16 = air_fit_low * water_fit_mid * corridors_fit_high * tourism_fit_low
    rule17 = air_fit_low * water_fit_mid * corridors_fit_high * tourism_fit_mid
    rule18 = air_fit_low * water_fit_mid * corridors_fit_high * tourism_fit_high
    rule19 = air_fit_low * water_fit_high * corridors_fit_low * tourism_fit_low
    rule20 = air_fit_low * water_fit_high * corridors_fit_low * tourism_fit_mid
    rule21 = air_fit_low * water_fit_high * corridors_fit_low * tourism_fit_high
    rule22 = air_fit_low * water_fit_high * corridors_fit_mid * tourism_fit_low
    rule23 = air_fit_low * water_fit_high * corridors_fit_mid * tourism_fit_mid
    rule24 = air_fit_low * water_fit_high * corridors_fit_mid * tourism_fit_high
    rule25 = air_fit_low * water_fit_high * corridors_fit_high * tourism_fit_low
    rule26 = air_fit_low * water_fit_high * corridors_fit_high * tourism_fit_mid
    rule27 = air_fit_low * water_fit_high * corridors_fit_high * tourism_fit_high
    rule28 = air_fit_mid * water_fit_low * corridors_fit_low * tourism_fit_low
    rule29 = air_fit_mid * water_fit_low * corridors_fit_low * tourism_fit_mid
    rule30 = air_fit_mid * water_fit_low * corridors_fit_low * tourism_fit_high
    rule31 = air_fit_mid * water_fit_low * corridors_fit_mid * tourism_fit_low
    rule32 = air_fit_mid * water_fit_low * corridors_fit_mid * tourism_fit_mid
    rule33 = air_fit_mid * water_fit_low * corridors_fit_mid * tourism_fit_high
    rule34 = air_fit_mid * water_fit_low * corridors_fit_high * tourism_fit_low
    rule35 = air_fit_mid * water_fit_low * corridors_fit_high * tourism_fit_mid
    rule36 = air_fit_mid * water_fit_low * corridors_fit_high * tourism_fit_high
    rule37 = air_fit_mid * water_fit_mid * corridors_fit_low * tourism_fit_low
    rule38 = air_fit_mid * water_fit_mid * corridors_fit_low * tourism_fit_mid
    rule39 = air_fit_mid * water_fit_mid * corridors_fit_low * tourism_fit_high
    rule40 = air_fit_mid * water_fit_mid * corridors_fit_mid * tourism_fit_low
    rule41 = air_fit_mid * water_fit_mid * corridors_fit_mid * tourism_fit_mid
    rule42 = air_fit_mid * water_fit_mid * corridors_fit_mid * tourism_fit_high
    rule43 = air_fit_mid * water_fit_mid * corridors_fit_high * tourism_fit_low
    rule44 = air_fit_mid * water_fit_mid * corridors_fit_high * tourism_fit_mid
    rule45 = air_fit_mid * water_fit_mid * corridors_fit_high * tourism_fit_high
    rule46 = air_fit_mid * water_fit_high * corridors_fit_low * tourism_fit_low
    rule47 = air_fit_mid * water_fit_high * corridors_fit_low * tourism_fit_mid
    rule48 = air_fit_mid * water_fit_high * corridors_fit_low * tourism_fit_high
    rule49 = air_fit_mid * water_fit_high * corridors_fit_mid * tourism_fit_low
    rule50 = air_fit_mid * water_fit_high * corridors_fit_mid * tourism_fit_mid
    rule51 = air_fit_mid * water_fit_high * corridors_fit_mid * tourism_fit_high
    rule52 = air_fit_mid * water_fit_high * corridors_fit_high * tourism_fit_low
    rule53 = air_fit_mid * water_fit_high * corridors_fit_high * tourism_fit_mid
    rule54 = air_fit_mid * water_fit_high * corridors_fit_high * tourism_fit_high
    rule55 = air_fit_high * water_fit_low * corridors_fit_low * tourism_fit_low
    rule56 = air_fit_high * water_fit_low * corridors_fit_low * tourism_fit_mid
    rule57 = air_fit_high * water_fit_low * corridors_fit_low * tourism_fit_high
    rule58 = air_fit_high * water_fit_low * corridors_fit_mid * tourism_fit_low
    rule59 = air_fit_high * water_fit_low * corridors_fit_mid * tourism_fit_mid
    rule60 = air_fit_high * water_fit_low * corridors_fit_mid * tourism_fit_high
    rule61 = air_fit_high * water_fit_low * corridors_fit_high * tourism_fit_low
    rule62 = air_fit_high * water_fit_low * corridors_fit_high * tourism_fit_mid
    rule63 = air_fit_high * water_fit_low * corridors_fit_high * tourism_fit_high
    rule64 = air_fit_high * water_fit_mid * corridors_fit_low * tourism_fit_low
    rule65 = air_fit_high * water_fit_mid * corridors_fit_low * tourism_fit_mid
    rule66 = air_fit_high * water_fit_mid * corridors_fit_low * tourism_fit_high
    rule67 = air_fit_high * water_fit_mid * corridors_fit_mid * tourism_fit_low
    rule68 = air_fit_high * water_fit_mid * corridors_fit_mid * tourism_fit_mid
    rule69 = air_fit_high * water_fit_mid * corridors_fit_mid * tourism_fit_high
    rule70 = air_fit_high * water_fit_mid * corridors_fit_high * tourism_fit_low
    rule71 = air_fit_high * water_fit_mid * corridors_fit_high * tourism_fit_mid
    rule72 = air_fit_high * water_fit_mid * corridors_fit_high * tourism_fit_high
    rule73 = air_fit_high * water_fit_high * corridors_fit_low * tourism_fit_low
    rule74 = air_fit_high * water_fit_high * corridors_fit_low * tourism_fit_mid
    rule75 = air_fit_high * water_fit_high * corridors_fit_low * tourism_fit_high
    rule76 = air_fit_high * water_fit_high * corridors_fit_mid * tourism_fit_low
    rule77 = air_fit_high * water_fit_high * corridors_fit_mid * tourism_fit_mid
    rule78 = air_fit_high * water_fit_high * corridors_fit_mid * tourism_fit_high
    rule79 = air_fit_high * water_fit_high * corridors_fit_high * tourism_fit_low
    rule80 = air_fit_high * water_fit_high * corridors_fit_high * tourism_fit_mid
    rule81 = air_fit_high * water_fit_high * corridors_fit_high * tourism_fit_high

    rules = [rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9, rule10,
    rule11, rule12, rule13, rule14, rule15, rule16, rule17, rule18, rule19, rule20,
    rule21, rule22, rule23, rule24, rule25, rule26, rule27, rule28, rule29, rule30,
    rule31, rule32, rule33, rule34, rule35, rule36, rule37, rule38, rule39, rule40,
    rule41, rule42, rule43, rule44, rule45, rule46, rule47, rule48, rule49, rule50,
    rule51, rule52, rule53, rule54, rule55, rule56, rule57, rule58, rule59, rule60,
    rule61, rule62, rule63, rule64, rule65, rule66, rule67, rule68, rule69, rule70,
    rule71, rule72, rule73, rule74, rule75, rule76, rule77, rule78, rule79, rule80,
    rule81]

    return rules

In [5]:
def fuzzification(rules):
    out_sustainable_tourism_low = np.fmax(
        np.fmax(np.fmax(np.fmax(np.fmax(
            np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
                np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
                    np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
                        np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
                            np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
                                np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
                                    np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
                                        rules[0], rules[1]), rules[2]), rules[3]), rules[4]), rules[5]),
                                        rules[6]), rules[7]), rules[8]), rules[9]), rules[10]),
                                        rules[11]), rules[12]), rules[14]), rules[17]), rules[18]),
                                        rules[19]), rules[20]), rules[22]), rules[23]), rules[26]),
                                        rules[27]), rules[28]), rules[29]), rules[32]), rules[35]),
                                        rules[38]), rules[41]), rules[44]), rules[45]), rules[48]),
                                        rules[51]), rules[54]), rules[57]), rules[60]), rules[63]),
                                        rules[64]), rules[66]), rules[69]), rules[70]), rules[73]),
                                        rules[76])
    
    out_sustainable_tourism_mid = np.fmax(np.fmax(
        np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
            np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
                np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
                    np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
                        np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
                            np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
                                np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
                                    rules[12], rules[13]), rules[15]), rules[16]), rules[21]),
                                    rules[24]), rules[25]), rules[30]), rules[31]), rules[33]),
                                    rules[34]), rules[36]), rules[37]), rules[40]), rules[41]),
                                    rules[43]), rules[44]), rules[47]), rules[48]), rules[50]),
                                    rules[51]), rules[53]), rules[54]), rules[56]), rules[57]),
                                    rules[59]), rules[60]), rules[62]), rules[63]), rules[65]),
                                    rules[67]), rules[68]), rules[71]), rules[72]), rules[74]),
                                    rules[75]), rules[77]), rules[78])
    
    out_sustainable_tourism_high = np.fmax(np.fmax(
        np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
            np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
                np.fmax(np.fmax(np.fmax(np.fmax(np.fmax(
                    rules[39], rules[42]), rules[45]), rules[49]), rules[51]),
                    rules[53]), rules[56]), rules[58]), rules[62]), rules[65]),
                    rules[67]), rules[69]), rules[71]), rules[74]), rules[75]),
                    rules[77]), rules[79]), rules[80])

    fuzzy_outputs = [out_sustainable_tourism_low, out_sustainable_tourism_mid, out_sustainable_tourism_high]
    
    return fuzzy_outputs

In [6]:
def defuzzification(fuzzy_outputs):
    # Defuzzification with centroid, bisector and mean of maximum methods
    fuzzy_out_sustainable_tourism = np.fmax(np.fmax(np.fmin(fuzzy_outputs[0], sustainable_tourism_low(x_sustainable_tourism)), 
                                            np.fmin(fuzzy_outputs[1], sustainable_tourism_mid(x_sustainable_tourism))),
                                            np.fmin(fuzzy_outputs[2], sustainable_tourism_high(x_sustainable_tourism)))
    
    defuzzified_sustainable_tourism  = fuzz.defuzz(x_sustainable_tourism, fuzzy_out_sustainable_tourism, 'centroid')

    return defuzzified_sustainable_tourism

In [7]:
def eval_fuzzy_system(input_air, input_water, input_corridors, input_tourism):
    rules = rules_base(input_air, input_water, input_corridors, input_tourism)
    fuzzy_outputs = fuzzification(rules)
    outputs = defuzzification(fuzzy_outputs)
    return outputs

In [8]:
results = {}

# Calculate the sustainable tourism indicator
for codigo_comuna, datos_comuna in commune_data.items():
    input_green_corridors = datos_comuna['Corredores verdes']
    input_air_quality = datos_comuna['Calidad de Aire']
    input_potable_water = datos_comuna['Producción Agua Potable']
    input_tourism = datos_comuna['Indicador turismo']
    
    # Execute fuzzy logic model
    sustainable_tourism = eval_fuzzy_system(input_air_quality, input_potable_water, input_green_corridors, input_tourism)
    
    # Save results
    results[codigo_comuna] = sustainable_tourism

results_list = [{'Código': codigo_comuna, 'Turismo sostenible': resultado} 
                for codigo_comuna, resultado in results.items()]

df_resultados = pd.DataFrame(results_list)
df_resultados.to_csv('Enviromental_tourism_model.csv', index=False)